In [ ]:
import pandas as pd  

In [ ]:
try:
    df = pd.read_csv('data.csv', sep=';', encoding='latin-1')
except Exception as e:
    print(f"Erreur lors du chargement du fichier: {e}")
    # Si le chargement échoue, nous arrêtons ici.
    raise

In [ ]:
df = df[df['Sexe'] != 'Sexe - total']

In [ ]:
all_Regions = df['Grande région'].unique()
allowed_regions = [region for region in all_Regions if region != 'Suisse']
print(f"Régions identifiées (excluant 'Suisse'): {len(allowed_regions)} - {allowed_regions}")

In [ ]:
df_for_each_region = {}
nombre_df = 0

for region in allowed_regions:
    # Filtrer le DataFrame principal pour la région actuelle et créer une copie propre (.copy())
    df_for_each_region[region] = df[df['Grande région'] == region].copy()
    nombre_df += 1

print(f"\n✅ Opération terminée. {nombre_df} DataFrames créés avec succès.")

In [ ]:
print("\n--- Aperçu des DataFrames créés ---")
for region, sub_df in df_for_each_region.items():
    # Afficher la taille de chaque nouveau DataFrame
    print(f"**{region}** : {sub_df.shape[0]} lignes, {sub_df.shape[1]} colonnes")
print("\n\n--- Exemples de données dans chaque DataFrame ---\n")
for region, sub_df in df_for_each_region.items():
    print(f"**{region}** : {sub_df.tail(2).to_markdown(index=False)}\n")
    
    # Afficher les 2 premières lignes du premier DataFrame pour démonstration
    # if region == allowed_regions[0]:
    #     print("Voici les 2 premières lignes du DataFrame pour la première région (Zurich) :")
    #     print(sub_df.head(2).to_markdown(index=False))
    #     print("...")

In [ ]:
import numpy as np

df_pay_gap_by_region = {}

cols_to_keep = [ 'Année', 'Grande région', 'Division économique',  'Position professionnelle']

for region, df_region in df_for_each_region.items():
   df_cleaned = df_region.replace(['..', ':', ';;', ''], np.nan)
   df_cleaned['Médiane'] = pd.to_numeric(df_cleaned['Médiane'], errors='coerce')
   df_pivot = df_cleaned.pivot_table(
       index=cols_to_keep,
       columns='Sexe',
       values='Médiane',
      aggfunc='first'
   ).reset_index()
   df_pivot.rename(columns={'Hommes': 'Mediane_Homme', 'Femmes': 'Mediane_Femme'}, inplace=True)
   df_pivot['Pay_Gap'] = df_pivot['Mediane_Homme'] - df_pivot['Mediane_Femme']
   final_cols = cols_to_keep + ['Mediane_Homme', 'Mediane_Femme', 'Pay_Gap']
   df_final = df_pivot[final_cols].copy()
   df_pay_gap_by_region[region] = df_final
   print(f"✅ Calcul du Pay Gap pour la région {region} terminé. Taille du DataFrame: {df_final.shape}")

In [ ]:
print("\n--- Aperçu du Résultat (Région de Zurich) ---")
df_zurich_diff = df_pay_gap_by_region['Zürich']
print(df_zurich_diff.tail().to_markdown(index=False))

In [ ]:
dfs_pay_gap_mean_by_region_year = []

print("Calcul de la moyenne de l'écart salarial (Médiane H-F) par Année et Région...")
for region, df_region in df_pay_gap_by_region.items():
    df_mean = df_region.groupby(['Année', 'Grande région'])['Pay_Gap'].mean().round(2).reset_index()
    df_mean.rename(columns={'Pay_Gap': 'Pay_Gap_Mean'}, inplace=True)
    dfs_pay_gap_mean_by_region_year.append(df_mean)
    print(f"✅ Moyenne calculée pour la région {region}. Taille du DataFrame: {df_mean.shape}")

In [ ]:
for df_mean in dfs_pay_gap_mean_by_region_year:
    print(f"{df_mean}\n")

In [ ]:
df_pay_gap = pd.concat(dfs_pay_gap_mean_by_region_year, ignore_index=True)

df_by_year = {}

annees_uniques = df_pay_gap['Année'].unique()

for annee in annees_uniques:
      df_annee = df_pay_gap[df_pay_gap['Année'] == annee].copy()
      df_by_year[annee] = df_annee

for annee, df_annee in df_by_year.items():
    print(f"{df_annee.to_markdown(index=False)}\n")

In [ ]:
df_to_print = {}
for annee, df_annee in df_by_year.items():
    region_pay_gap = df_annee[['Grande région', 'Pay_Gap_Mean']].copy()
    df_to_print[annee] = region_pay_gap.set_index('Grande région')['Pay_Gap_Mean']

for df_annee in df_to_print.values():
    print(f"{df_annee.to_markdown(index=True)}\n")